# Training file Hotel Id assignment #
*more info here*

# Imports, Setup, etc. #

In [ ]:
# Imports:

! pip install timm --no-index --find-links=file:///kaggle/input/timm-zipped/timm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import timm #includes many pre-build models & tools.


# Don't know what this does, so I'll just keep it there just in case...
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# File & Folder names (not all used atm):        
PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
DATA_FOLDER = "../input/hotelid-2022-train-images-256x256/"
IMAGE_FOLDER = DATA_FOLDER + "images/"
OUTPUT_FOLDER = ""


## Image Processing

In [ ]:

# Expecting Data sets and data loaders here!


## Auxiliariy functions

In [ ]:
def save_checkpoint(model, scheduler, optimizer, epoch, name, loss=None, score=None):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }

    torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")



DataSet

In [ ]:
class HotelDataset(Dataset):
    #Have to define a path parameter
    
    def __init__(self, root_dir="../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/", data_path="train_images/", min_images=1, max_images=None, max_entries=None, transform=None):
        MAX_WINDOWS_FILENAME_CHAR_LENGTH = 260
        self.root_dir = root_dir
        self.data_path = data_path
        self.transform = transform
        self.full_data_path = os.path.join(root_dir, data_path)
        
        #We expect the following structure:
        #full_data_path contains only folders, each folder representing a hotel, and having an unique name
        #inside the hotel folders there are only image files
        
        dirs = os.listdir(self.full_data_path)
        self.num_labels = len(dirs)
        self.total_files = 0
        for directory in dirs:
            files=os.listdir(os.path.join(self.full_data_path,directory))
            files_number = len(files)
            if max_images != None and files_number > max_images:
                dirs.remove(directory)
                continue
            if files_number < min_images:
                dirs.remove(directory)
                continue
            self.total_files += files_number
            if max_entries != None and max_entries >= self.total_files:
                dirs = dirs[:max_entries]
                break
        
        self.hotel_data = np.chararray([self.total_files,2], itemsize=MAX_WINDOWS_FILENAME_CHAR_LENGTH)
        iterator = 0
        for directory in dirs:
            files=os.listdir(os.path.join(self.full_data_path,directory))
            for f in files:
                self.hotel_data[iterator] = [f, directory]
            
    def __getitem__(self, index):
        label = self.hotel_data[index,1].decode()
        hotel_image_id = self.hotel_data[index,0].decode()
        image_path = os.path.join(self.full_data_path, label, hotel_image_id)
        
        image = Image.open(image_path)
        item = image
        if self.transform != None:
            item = self.transform(image)
        return item, label
        
    def __len__(self):
        return self.total_files

## Model

In [ ]:
# Define model:

class Hotel_Model (nn.Module):
    
    def __init__(self, n_classes = 100, embeddings_size = 64):
        super(EmbeddingModel, self).__init__()
        
        # use existing timm-model for bulk of work:
        self.main_model = timm.create_model(backbone_name, num_classes=n_classes, pretrained=True)
        
        # Use own torch-layers for last embedding step and classification:
        in_features = self.main_model.get_classifier().infeatures        
        self.main_model.classifier = nn.Identity # turn off classifier in pre-trained model
        self.embedding = nn.Linear(in_features, embedding_size)
        
        #Add own final layer and classifier !!! CHANGE THIS TO RETRIEVAL !!!
        self.classifier = nn.Linear(embedding_size, n_classes) 
        

    def embed_and_classify(self,x):
        x = self.forward(x)
        return x, self.classifier(x)
    
    def forward(self,x):
        x = self.main_model(x)
        x = x.view(x.size(0), -1) #re-scale
        x = self.embedding(x)
        return x

## Training Cycle

In [ ]:
def train_epoch(train_loader, optimizer, loss_fn, model):
    """Training and adjusting model for one epoch."""
    
    for i, data in enumerate(train_loader): #loop over all data in current batch (in loader!)        
        # Initialisation:
        inputs, labels =  # Unpack data
        optimizer = zero_grad()
        
        # Make prediction, compute loss
        prediction = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backwards()
        
        # Adjust model 
        optimizer.step()
        
        # Gather data and stuff: TBW

def val_epoch(val_loader, loss_fn, model):
    "Compute validation data for one epoch"
    
    # Just like train_epoch, but without adjusting the model...
    total_vloss = 0.0
    
    for i, vdata in enumerate(val_loader):
        inputs, labels = vdata
        prediction = model(inputs)
        vloss = loss_fn(prediction, labels)
        total_vloss += vloss
        
    return total_vloss / (i+1)

In [ ]:
def train_complete(train_loader, val_loader, model, optimizer, loss_fn, nmbr_epochs = 5):
    """Trains and Validates model given loaders and initial model."""

    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    best_vloss = 10**9
    for epoch in range(nmbr_epochs):
        
        # Train one epoch:
        model.train(True)
        train_epoch(train_loader, optimizer, loss_fn, model)
        
        # Validate epoch:
        model.train(False)
        vloss = val_epoch(val_loader,loss_fn, model)
        
        # Save current model:
        save_checkpoint(model)        

In [ ]:
def predict(test_loader, model):
    """Function to predict the 5 most likely hotels"""
    
    # For now, this is just the most likely hotel, 5 times
    nmbr_predictions = 1 # Part of loader I assume?
    predictions = []
    for i,input in enumerate(test_loader):
        prediction = model(input)
        #print(prediction) # I'm guessing this is just one prediction, not probs for all. If so, how do we get these?
        predictions.append(prediction)
    
    return predictions

## Running the code & Making submission:

In [ ]:
# Define model & Datasets
model = Hotel_Model()
train_loader, val_loader, test_loader = ... #TBW

# Optimizer & Loss function: Placeholders for now:

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)

loss_fn = torch.nn.CrossEntropyLoss

In [ ]:
# Train model:

nmbr_epochs = 1
train_complete(train_loader, val_loader, model, optimizer, loss_fn, nmbr_epochs)

# Make prediction:

final_preds = predict(test_loader, model)
final_preds = final_preds # Transform to correct naming: these differ slightly I though, Nicolay?

final_preds.to_csv(OUTPUT_FOLDER + 'submission.csv', index=False)